# Tweets classification by its hashtags and no labeled tweets hashtag predication

> Using top n hashtags as label to build a supervised model for tweets classification and hashtag predication

In [1]:
#import tensorflow backend
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12178098609830036301
]


In [2]:
import keras
from keras.layers import *
from keras.models import Sequential, Model
from keras.optimizers import *
from keras.preprocessing import sequence

print(keras.__version__)
print(keras.backend.backend())

Using TensorFlow backend.


2.0.8
tensorflow


In [3]:
import pandas as pd
import numpy as np
import bcolz
import pickle

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
    
def load_array(fname):
    return bcolz.open(fname)[:]

def save_dict(fname, dictionary):
    with open(fname, 'wb') as f:
        pickle.dump(dictionary, f)

def load_dict(fname):
    with open(fname, 'rb') as f:
        return pickle.load(f)

In [4]:
#load processed word enbeddings
path = 'wordsenbeddings/'
res_path = path + 'results/'

def load_vectors(name):
    loc = res_path + name
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))

In [5]:
#load tweets data
tweets_file = "temp/tweets4classification.csv"
preprocessed_tweets = pd.read_csv(tweets_file)
preprocessed_tweets.head(5)

,id,text,raw,orignal_hashtags,hashtags,words
0,418263863772327936,rt : hpv vax coverage could be 93% if doctors ...,rt @cdcstd: #hpv vax coverage could be 93% if ...,"['#hpv', '#vaccine']","['hpv', 'vaccin']","['rt', ':', 'hpv', 'vax', 'coverage', 'could',..."
1,418263863772327936,rt : hpv vax coverage could be 93% if doctors ...,rt @cdcstd: #hpv vax coverage could be 93% if ...,"['#hpv', '#vaccine']","['hpv', 'vaccin']","['rt', ':', 'hpv', 'vax', 'coverage', 'could',..."
2,418194978310209536,rt : hpv vax coverage could be 93% if doctors ...,rt @cdcstd: #hpv vax coverage could be 93% if ...,"['#hpv', '#vaccine']","['hpv', 'vaccin']","['rt', ':', 'hpv', 'vax', 'coverage', 'could',..."
3,418194978310209536,rt : hpv vax coverage could be 93% if doctors ...,rt @cdcstd: #hpv vax coverage could be 93% if ...,"['#hpv', '#vaccine']","['hpv', 'vaccin']","['rt', ':', 'hpv', 'vax', 'coverage', 'could',..."
4,418216659690991616,rt : hpv vax coverage could be 93% if doctors ...,rt @cdcstd: #hpv vax coverage could be 93% if ...,"['#hpv', '#vaccine']","['hpv', 'vaccin']","['rt', ':', 'hpv', 'vax', 'coverage', 'could',..."
